In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-27"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14105,2024-10-27,Brasil Nbb,20:00,Fortaleza B.C.,Botafogo,1.31,3.20,148.5,1.83,1.83,-7.5,1.94,1.77,U3xCSXgR,0.763359,0.312500,0.546448,0.546448,0.075859,0.0,0.2,NaN,NaN,115.628,26.245867,0.226985,266.124,184.872881,0.694687,98.192,272.310,85.40,186.48,0.0,0.0,0.0,0.0,0.592653,0.000000,0.064802,-0.79,-0.158,-1.962025,0.673648,0.6,-0.073648,3.51,0.702,3.133903,0.288438,0.5,0.211562
14106,2024-10-27,Espanha Acb,08:30,Real Madrid,Manresa,1.13,6.20,168.5,1.88,1.92,-15.5,2.03,1.66,fsjt4ukS,0.884956,0.161290,0.531915,0.520833,0.046246,0.4,0.2,NaN,NaN,127.488,46.794658,0.367051,171.018,28.733862,0.168017,102.150,163.178,103.55,140.13,0.0,0.0,0.0,0.0,0.978180,0.014886,0.141805,0.53,0.106,1.226415,0.000000,0.0,0.000000,-1.11,-0.222,-23.423423,0.424791,0.5,0.075209
14107,2024-10-27,Espanha Acb,09:00,Tenerife,Gran Canaria,1.46,2.74,160.5,1.80,1.94,-6.5,2.04,1.72,ERWCbJCk,0.684932,0.364964,0.555556,0.515464,0.049895,0.8,0.0,NaN,NaN,162.078,40.492429,0.249833,112.314,15.923846,0.141780,167.452,121.062,230.23,127.30,0.0,0.0,0.0,0.0,0.430998,0.052938,0.120359,1.87,0.374,1.229947,0.629779,0.6,-0.029779,-3.42,-0.684,-2.543860,0.495396,0.4,-0.095396
14108,2024-10-27,Espanha Acb,13:00,Bilbao,Valencia,2.36,1.60,163.5,1.85,1.89,2.5,1.99,1.76,nuVKdcs2,0.423729,0.625000,0.540541,0.529101,0.048729,0.4,0.6,NaN,NaN,214.702,84.393170,0.393071,164.286,50.138244,0.305189,206.128,157.892,351.09,124.41,1.0,0.0,0.0,0.0,0.271415,0.015125,0.086738,7.15,1.430,0.951049,0.520896,0.8,0.279104,-1.33,-0.266,-2.255639,0.487060,0.6,0.112940
14109,2024-10-27,Espanha Acb,13:00,Breogan,Lleida,1.53,2.53,160.5,1.84,1.90,-5.5,2.02,1.74,QNZaK0Z8,0.653595,0.395257,0.543478,0.526316,0.048852,0.6,0.0,NaN,NaN,188.202,63.276562,0.336216,NaN,NaN,NaN,213.114,0.000,214.50,529.20,0.0,1.0,0.0,0.0,0.348328,0.022688,0.105314,-1.92,-0.384,-1.380208,0.484468,0.4,-0.084468,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14273,2024-10-27,México Lnbp,21:00,Diablos Rojos,Dorados,1.22,3.68,175.5,1.86,1.80,-9.5,2.05,1.65,K4J04QxS,0.819672,0.271739,0.537634,0.555556,0.091411,0.0,0.0,NaN,NaN,NaN,NaN,NaN,136.052,28.033207,0.206048,0.000,0.000,113.05,0.00,0.0,0.0,0.0,0.0,0.709993,0.023184,0.152888,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
14274,2024-10-27,Polônia 1. Liga,14:00,Starogard Gdanski,Pelplin,1.44,2.52,158.5,1.80,1.86,-6.5,2.05,1.65,Ui23pYQs,0.694444,0.396825,0.555556,0.537634,0.091270,0.4,0.6,NaN,NaN,167.040,34.042106,0.203796,125.854,40.115207,0.318744,123.112,145.992,166.86,98.80,0.0,0.0,0.0,0.0,0.385695,0.023184,0.152888,-0.07,-0.014,-31.428571,0.729934,0.6,-0.129934,-0.34,-0.068,-22.352941,0.565397,0.6,0.034603
14275,2024-10-27,Reino Unido Troféu Da Slb,12:00,Manchester Basketball,Sheffield Sharks,4.31,1.16,170.5,1.83,1.83,8.5,2.03,1.67,CS7F8WKF,0.232019,0.862069,0.546448,0.546448,0.094088,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.814401,0.000000,0.137599,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
14276,2024-10-27,Romênia Liga Nacional Feminina,13:00,Arad F,Sepsi S. F,3.07,1.30,136.5,1.83,1.83,6.5,2.02,1.67,f3yD7W44,0.325733,0.769231,0.546448,0.546448,0.094964,0.0,0.0,NaN,NaN,97.158,19.866253,0.204474,NaN,NaN,NaN,0.0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,19:00,Eua Nba,Brooklyn Nets,Milwaukee Bucks,229.5,1.80,0.6178,Over
1,21:30,Eua Nba,Golden State Warriors,Los Angeles Clippers,220.5,1.85,0.9043,Over
